# Adaptive Cost Sensitive Trees Expriment

## imports 

In [1]:
from ada_csl_wrc.evaluation import evaluate
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import train_test_split

from ada_csl_wrc.models import CostSensitiveDecisionTreeClassifier

import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif
from ada_csl_wrc.models import ConstrainedCSDecisionTree
from ada_csl_wrc.models import Constrained
from sklearn.model_selection import train_test_split
from ada_csl_wrc.utils import prediction_up_to_constraint
from ada_csl_wrc.utils import prepare_for_cost_cle
from ada_csl_wrc.utils import filter_only_worst_features
from ada_csl_wrc.utils import find_effective_threshold
import json
import pickle
from copy import deepcopy

import numbers
from sklearn.utils import check_random_state, column_or_1d
from six import with_metaclass
from abc import ABCMeta, abstractmethod
from sklearn.ensemble import BaseEnsemble
from sklearn.base import ClassifierMixin
from joblib import Parallel, delayed, cpu_count
from sklearn.utils.random import sample_without_replacement
import itertools

from timeit import default_timer
import uuid
import os

loaded ada_csl_wrc.utils


## Random Forest Cost Sensitive Model Code 
### copied from https://github.com/albahnsen/CostSensitiveClassification/tree/master/costcla/models

In [2]:
def savings_score(y_true, y_pred, cost_mat):
    #TODO: update description
    """Savings score.

    This function calculates the savings cost of using y_pred on y_true with
    cost-matrix cost-mat, as the difference of y_pred and the cost_loss of a naive
    classification model.

    Parameters
    ----------
    y_true : array-like or label indicator matrix
        Ground truth (correct) labels.

    y_pred : array-like or label indicator matrix
        Predicted labels, as returned by a classifier.

    cost_mat : array-like of shape = [n_samples, 4]
        Cost matrix of the classification problem
        Where the columns represents the costs of: false positives, false negatives,
        true positives and true negatives, for each example.

    Returns
    -------
    score : float
        Savings of a using y_pred on y_true with cost-matrix cost-mat

        The best performance is 1.

    References
    ----------
    .. [1] A. Correa Bahnsen, A. Stojanovic, D.Aouada, B, Ottersten,
           `"Improving Credit Card Fraud Detection with Calibrated Probabilities" <http://albahnsen.com/files/%20Improving%20Credit%20Card%20Fraud%20Detection%20by%20using%20Calibrated%20Probabilities%20-%20Publish.pdf>`__, in Proceedings of the fourteenth SIAM International Conference on Data Mining,
           677-685, 2014.

    See also
    --------
    cost_loss

    Examples
    --------
    >>> import numpy as np
    >>> from costcla.metrics import savings_score, cost_loss
    >>> y_pred = [0, 1, 0, 0]
    >>> y_true = [0, 1, 1, 0]
    >>> cost_mat = np.array([[4, 1, 0, 0], [1, 3, 0, 0], [2, 3, 0, 0], [2, 1, 0, 0]])
    >>> savings_score(y_true, y_pred, cost_mat)
    0.5
    """

    #TODO: Check consistency of cost_mat
    y_true = column_or_1d(y_true)
    y_pred = column_or_1d(y_pred)
    n_samples = len(y_true)

    # Calculate the cost of naive prediction
    cost_base = min(cost_loss(y_true, np.zeros(n_samples), cost_mat),
                    cost_loss(y_true, np.ones(n_samples), cost_mat))

    cost = cost_loss(y_true, y_pred, cost_mat)
    return 1.0 - cost / cost_base

In [3]:
def cost_loss(y_true, y_pred, cost_mat):
    #TODO: update description
    """Cost classification loss.

    This function calculates the cost of using y_pred on y_true with
    cost-matrix cost-mat. It differ from traditional classification evaluation
    measures since measures such as accuracy asing the same cost to different
    errors, but that is not the real case in several real-world classification
    problems as they are example-dependent cost-sensitive in nature, where the
    costs due to misclassification vary between examples.

    Parameters
    ----------
    y_true : array-like or label indicator matrix
        Ground truth (correct) labels.

    y_pred : array-like or label indicator matrix
        Predicted labels, as returned by a classifier.

    cost_mat : array-like of shape = [n_samples, 4]
        Cost matrix of the classification problem
        Where the columns represents the costs of: false positives, false negatives,
        true positives and true negatives, for each example.

    Returns
    -------
    loss : float
        Cost of a using y_pred on y_true with cost-matrix cost-mat

    References
    ----------
    .. [1] C. Elkan, "The foundations of Cost-Sensitive Learning",
           in Seventeenth International Joint Conference on Artificial Intelligence,
           973-978, 2001.

    .. [2] A. Correa Bahnsen, A. Stojanovic, D.Aouada, B, Ottersten,
           `"Improving Credit Card Fraud Detection with Calibrated Probabilities" <http://albahnsen.com/files/%20Improving%20Credit%20Card%20Fraud%20Detection%20by%20using%20Calibrated%20Probabilities%20-%20Publish.pdf>`__, in Proceedings of the fourteenth SIAM International Conference on Data Mining,
           677-685, 2014.

    See also
    --------
    savings_score

    Examples
    --------
    >>> import numpy as np
    >>> from costcla.metrics import cost_loss
    >>> y_pred = [0, 1, 0, 0]
    >>> y_true = [0, 1, 1, 0]
    >>> cost_mat = np.array([[4, 1, 0, 0], [1, 3, 0, 0], [2, 3, 0, 0], [2, 1, 0, 0]])
    >>> cost_loss(y_true, y_pred, cost_mat)
    3
    """

    #TODO: Check consistency of cost_mat

    y_true = column_or_1d(y_true)
    y_true = (y_true == 1).astype(np.float32)
    y_pred = column_or_1d(y_pred)
    y_pred = (y_pred == 1).astype(np.float32)
    cost = y_true * ((1 - y_pred) * cost_mat[:, 1] + y_pred * cost_mat[:, 2])
    cost += (1 - y_true) * (y_pred * cost_mat[:, 0] + (1 - y_pred) * cost_mat[:, 3])
    return np.sum(cost)

In [4]:
MAX_INT = np.iinfo(np.int32).max
class BaseBagging(with_metaclass(ABCMeta, BaseEnsemble)):
    """Base class for Bagging meta-estimator.

    Warning: This class should not be used directly. Use derived classes
    instead.
    """

    @abstractmethod
    def __init__(self,
                 base_estimator=None,
                 n_estimators=10,
                 max_samples=1.0,
                 max_features=1.0,
                 bootstrap=True,
                 bootstrap_features=False,
                 combination='majority_voting',
                 n_jobs=1,
                 random_state=None,
                 verbose=0):
        super(BaseBagging, self).__init__(
            base_estimator=base_estimator,
            n_estimators=n_estimators)

        self.max_samples = max_samples
        self.max_features = max_features
        self.bootstrap = bootstrap
        self.bootstrap_features = bootstrap_features
        self.combination = combination
        self.n_jobs = n_jobs
        self.random_state = random_state
        self.verbose = verbose

    def fit(self, X, y, cost_mat, sample_weight=None):
        """Build a Bagging ensemble of estimators from the training set (X, y).

        Parameters
        ----------
        X : {array-like, sparse matrix} of shape = [n_samples, n_features]
            The training input samples. Sparse matrices are accepted only if
            they are supported by the base estimator.

        y : array-like, shape = [n_samples]
            The target values (class labels in classification, real numbers in
            regression).

        cost_mat : array-like of shape = [n_samples, 4]
            Cost matrix of the classification problem
            Where the columns represents the costs of: false positives, false negatives,
            true positives and true negatives, for each example.

        sample_weight : array-like, shape = [n_samples] or None
            Sample weights. If None, then samples are equally weighted.
            Note that this is supported only if the base estimator supports
            sample weighting.

        Returns
        -------
        self : object
            Returns self.
        """
        random_state = check_random_state(self.random_state)

        # Convert data
        # X, y = check_X_y(X, y, ['csr', 'csc', 'coo'])  # Not in sklearn verion 0.15

        # Remap output
        n_samples, self.n_features_ = X.shape
        y = self._validate_y(y)

        # Check parameters
        self._validate_estimator()

        if isinstance(self.max_samples, (numbers.Integral, np.integer)):
            max_samples = self.max_samples
        else:  # float
            max_samples = int(self.max_samples * X.shape[0])

        if not (0 < max_samples <= X.shape[0]):
            raise ValueError("max_samples must be in (0, n_samples]")

        if isinstance(self.max_features, (numbers.Integral, np.integer)):
            max_features = self.max_features
        else:  # float
            max_features = int(self.max_features * self.n_features_)

        if not (0 < max_features <= self.n_features_):
            raise ValueError("max_features must be in (0, n_features]")

        # Free allocated memory, if any
        self.estimators_ = None

        # Parallel loop
        n_jobs, n_estimators, starts = _partition_estimators(self.n_estimators,
                                                             self.n_jobs)
        
        seeds = random_state.randint(MAX_INT, size=self.n_estimators)

        all_results = Parallel(n_jobs=n_jobs, verbose=self.verbose)(
            delayed(_parallel_build_estimators)(
                n_estimators[i],
                self,
                X,
                y,
                cost_mat,
                seeds[starts[i]:starts[i + 1]],
                verbose=self.verbose)
            for i in range(n_jobs))

        # Reduce
        self.estimators_ = list(itertools.chain.from_iterable(
            t[0] for t in all_results))
        self.estimators_samples_ = list(itertools.chain.from_iterable(
            t[1] for t in all_results))
        self.estimators_features_ = list(itertools.chain.from_iterable(
            t[2] for t in all_results))

        self._evaluate_oob_savings(X, y, cost_mat)

        if self.combination in ['stacking', 'stacking_proba', 'stacking_bmr', 'stacking_proba_bmr']:
            self._fit_stacking_model(X, y, cost_mat)

        if self.combination in ['majority_bmr', 'weighted_bmr', 'stacking_bmr', 'stacking_proba_bmr']:
            self._fit_bmr_model(X, y)

        return self

    def _fit_bmr_model(self, X, y):
        """Private function used to fit the BayesMinimumRisk model."""
        self.f_bmr = BayesMinimumRiskClassifier()
        X_bmr = self.predict_proba(X)
        self.f_bmr.fit(y, X_bmr)
        return self

    def _fit_stacking_model(self, X, y, cost_mat, max_iter=100):
        """Private function used to fit the stacking model."""
        self.f_staking = CostSensitiveLogisticRegression(
            verbose=self.verbose, max_iter=max_iter)
        X_stacking = _create_stacking_set(self.estimators_, self.estimators_features_,
                                          self.estimators_weight_, X, self.combination)
        self.f_staking.fit(X_stacking, y, cost_mat)
        return self

    #TODO: _evaluate_oob_savings in parallel
    def _evaluate_oob_savings(self, X, y, cost_mat):
        """Private function used to calculate the OOB Savings of each estimator."""
        estimators_weight = []
        for estimator, samples, features in zip(self.estimators_, self.estimators_samples_,
                                                self.estimators_features_):
            # Test if all examples where used for training
            if not np.any(~samples):
                # Then use training
                oob_pred = estimator.predict(X[:, features])
                oob_savings = max(0, savings_score(y, oob_pred, cost_mat))
            else:
                # Then use OOB
                oob_pred = estimator.predict((X[~samples])[:, features])
                oob_savings = max(0, savings_score(
                    y[~samples], oob_pred, cost_mat[~samples]))

            estimators_weight.append(oob_savings)

        # Control in case were all weights are 0
        if sum(estimators_weight) == 0:
            self.estimators_weight_ = np.ones(
                len(estimators_weight)) / len(estimators_weight)
        else:
            self.estimators_weight_ = (
                np.array(estimators_weight) / sum(estimators_weight)).tolist()

        return self

    def _validate_y(self, y):
        # Default implementation
        return column_or_1d(y, warn=True)

In [5]:
def _partition_estimators(n_estimators, n_jobs):
    """Private function used to partition estimators between jobs."""
    # Compute the number of jobs
    if n_jobs == -1:
        n_jobs = min(cpu_count(), n_estimators)

    else:
        n_jobs = min(n_jobs, n_estimators)

    # Partition estimators between jobs
    n_estimators_per_job = (n_estimators // n_jobs) * np.ones(n_jobs,
                                                              dtype=np.int64)
    n_estimators_per_job[:n_estimators % n_jobs] += 1
    starts = np.cumsum(n_estimators_per_job)

    return n_jobs, n_estimators_per_job.tolist(), [0] + starts.tolist()

In [123]:
def _parallel_build_estimators(n_estimators, ensemble, X, y, cost_mat,
                               seeds, verbose,min_samples_leaf, max_depth):#Chen Added min_samples_leaf and max_depth
    """Private function used to build a batch of estimators within a job."""
    # Retrieve settings
    n_samples, n_features = X.shape
    max_samples = ensemble.max_samples
    max_features = ensemble.max_features

    if (not isinstance(max_samples, (numbers.Integral, np.integer)) and
            (0.0 < max_samples <= 1.0)):
        max_samples = int(max_samples * n_samples)

    if (not isinstance(max_features, (numbers.Integral, np.integer)) and
            (0.0 < max_features <= 1.0)):
        max_features = int(max_features * n_features)

    bootstrap = ensemble.bootstrap
    bootstrap_features = ensemble.bootstrap_features

    # Build estimators
    estimators = []
    estimators_samples = []
    estimators_features = []

    for i in range(n_estimators):
        if verbose > 1:
            print(("building estimator %d of %d" % (i + 1, n_estimators)))

        random_state = check_random_state(seeds[i])
        seed = check_random_state(random_state.randint(MAX_INT))
        estimator = ensemble._make_estimator(append=False)

        try:  # Not all estimator accept a random_state
            estimator.set_params(min_samples_leaf = min_samples_leaf, max_depth = max_depth, random_state=seed)#Chen Added min_samples_leaf and max_depth
        except ValueError:
            pass

        # Draw features
        if bootstrap_features:
            features = random_state.randint(0, n_features, max_features)
        else:
            features = sample_without_replacement(n_features,
                                                  max_features,
                                                  random_state=random_state)

        # Draw samples, using a mask, and then fit
        if bootstrap:
            indices = random_state.randint(0, n_samples, max_samples)
        else:
            indices = sample_without_replacement(n_samples,
                                                 max_samples,
                                                 random_state=random_state)

        sample_counts = np.bincount(indices, minlength=n_samples)

        estimator.fit((X[indices])[:, features],
                      y[indices], cost_mat[indices, :])
        samples = sample_counts > 0.

        estimators.append(estimator)
        estimators_samples.append(samples)
        estimators_features.append(features)

    return estimators, estimators_samples, estimators_features

In [7]:
def _parallel_predict_proba(estimators, estimators_features, X, n_classes, combination, estimators_weight):
    """Private function used to compute (proba-)predictions within a job."""
    n_samples = X.shape[0]
    proba = np.zeros((n_samples, n_classes))

    for estimator, features, weight in zip(estimators, estimators_features, estimators_weight):
        proba_estimator = estimator.predict_proba(X[:, features])
        if combination in ['weighted_voting', 'weighted_bmr']:
            proba += proba_estimator * weight
        else:
            proba += proba_estimator

    return proba

In [8]:
def _parallel_predict(estimators, estimators_features, X, n_classes, combination, estimators_weight):
    """Private function used to compute predictions within a job."""
    n_samples = X.shape[0]
    pred = np.zeros((n_samples, n_classes))
    n_estimators = len(estimators)

    for estimator, features, weight in zip(estimators, estimators_features, estimators_weight):
        # Resort to voting
        predictions = estimator.predict(X[:, features])

        for i in range(n_samples):
            if combination == 'weighted_voting':
                pred[i, int(predictions[i])] += 1 * weight
            else:
                pred[i, int(predictions[i])] += 1

    return pred

In [9]:

class BaggingClassifier(BaseBagging, ClassifierMixin):
    """A Bagging classifier.

    A Bagging classifier is an ensemble meta-estimator that fits base
    classifiers each on random subsets of the original dataset and then
    aggregate their individual predictions (either by voting or by averaging)
    to form a final prediction. Such a meta-estimator can typically be used as
    a way to reduce the variance of a black-box estimator (e.g., a decision
    tree), by introducing randomization into its construction procedure and
    then making an ensemble out of it.

    This algorithm encompasses several works from the literature. When random
    subsets of the dataset are drawn as random subsets of the samples, then
    this algorithm is known as Pasting [1]_. If samples are drawn with
    replacement, then the method is known as Bagging [2]_. When random subsets
    of the dataset are drawn as random subsets of the features, then the method
    is known as Random Subspaces [3]_. Finally, when base estimators are built
    on subsets of both samples and features, then the method is known as
    Random Patches [4]_.

    Parameters
    ----------
    base_estimator : object or None, optional (default=None)
        The base estimator to fit on random subsets of the dataset.
        If None, then the base estimator is a decision tree.

    n_estimators : int, optional (default=10)
        The number of base estimators in the ensemble.

    max_samples : int or float, optional (default=1.0)
        The number of samples to draw from X to train each base estimator.
            - If int, then draw `max_samples` samples.
            - If float, then draw `max_samples * X.shape[0]` samples.

    max_features : int or float, optional (default=1.0)
        The number of features to draw from X to train each base estimator.
            - If int, then draw `max_features` features.
            - If float, then draw `max_features * X.shape[1]` features.

    bootstrap : boolean, optional (default=True)
        Whether samples are drawn with replacement.

    bootstrap_features : boolean, optional (default=False)
        Whether features are drawn with replacement.

    combination : string, optional (default="majority_voting")
        Which combination method to use:
          - If "majority_voting" then combine by majority voting
          - If "weighted_voting" then combine by weighted voting using the
            out of bag savings as the weight for each estimator.
          - If "stacking" then a Cost Sensitive Logistic Regression is used
            to learn the combination.
          - If "stacking_proba" then a Cost Sensitive Logistic Regression trained
            with the estimated probabilities is used to learn the combination,.
          - If "stacking_bmr" then a Cost Sensitive Logistic Regression is used
            to learn the probabilities and a BayesMinimumRisk for the prediction.
          - If "stacking_proba_bmr" then a Cost Sensitive Logistic Regression trained
            with the estimated probabilities is used to learn the probabilities,
            and a BayesMinimumRisk for the prediction.
          - If "majority_bmr" then the BayesMinimumRisk algorithm is used to make the
            prediction using the predicted probabilities of majority_voting
          - If "weighted_bmr" then the BayesMinimumRisk algorithm is used to make the
            prediction using the predicted probabilities of weighted_voting

    n_jobs : int, optional (default=1)
        The number of jobs to run in parallel for both `fit` and `predict`.
        If -1, then the number of jobs is set to the number of cores.

    random_state : int, RandomState instance or None, optional (default=None)
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`.

    verbose : int, optional (default=0)
        Controls the verbosity of the building process.

    Attributes
    ----------
    `base_estimator_`: list of estimators
        The base estimator from which the ensemble is grown.

    `estimators_`: list of estimators
        The collection of fitted base estimators.

    `estimators_samples_`: list of arrays
        The subset of drawn samples (i.e., the in-bag samples) for each base
        estimator.

    `estimators_features_`: list of arrays
        The subset of drawn features for each base estimator.

    `classes_`: array of shape = [n_classes]
        The classes labels.

    `n_classes_`: int or list
        The number of classes.

    References
    ----------

    .. [1] L. Breiman, "Pasting small votes for classification in large
           databases and on-line", Machine Learning, 36(1), 85-103, 1999.

    .. [2] L. Breiman, "Bagging predictors", Machine Learning, 24(2), 123-140,
           1996.

    .. [3] T. Ho, "The random subspace method for constructing decision
           forests", Pattern Analysis and Machine Intelligence, 20(8), 832-844,
           1998.

    .. [4] G. Louppe and P. Geurts, "Ensembles on Random Patches", Machine
           Learning and Knowledge Discovery in Databases, 346-361, 2012.
    """

    def __init__(self,
                 base_estimator=None,
                 n_estimators=10,
                 max_samples=1.0,
                 max_features=1.0,
                 bootstrap=True,
                 bootstrap_features=False,
                 combination='voting',
                 n_jobs=1,
                 random_state=None,
                 verbose=0):

        super(BaggingClassifier, self).__init__(
            base_estimator,
            n_estimators=n_estimators,
            max_samples=max_samples,
            max_features=max_features,
            bootstrap=bootstrap,
            bootstrap_features=bootstrap_features,
            combination=combination,
            n_jobs=n_jobs,
            random_state=random_state,
            verbose=verbose)

    def _validate_estimator(self):
        """Check the estimator and set the base_estimator_ attribute."""
        super(BaggingClassifier, self)._validate_estimator(
            default=DecisionTreeClassifier())

    def _validate_y(self, y):
        y = column_or_1d(y, warn=True)
        self.classes_, y = np.unique(y, return_inverse=True)
        self.n_classes_ = len(self.classes_)

        return y

    def predict(self, X, cost_mat=None):
        """Predict class for X.

        The predicted class of an input sample is computed as the class with
        the highest mean predicted probability. If base estimators do not
        implement a ``predict_proba`` method, then it resorts to voting.

        Parameters
        ----------
        X : {array-like, sparse matrix} of shape = [n_samples, n_features]
            The training input samples. Sparse matrices are accepted only if
            they are supported by the base estimator.

        cost_mat : optional array-like of shape = [n_samples, 4], (default=None)
            Cost matrix of the classification problem
            Where the columns represents the costs of: false positives, false negatives,
            true positives and true negatives, for each example.

        Returns
        -------
        pred : array of shape = [n_samples]
            The predicted classes.
        """
        # Check data
        # X = check_array(X, accept_sparse=['csr', 'csc', 'coo'])  # Dont in version 0.15

        if self.n_features_ != X.shape[1]:
            raise ValueError("Number of features of the model must "
                             "match the input. Model n_features is {0} and "
                             "input n_features is {1}."
                             "".format(self.n_features_, X.shape[1]))

        # TODO: check if combination in possible combinations

        if self.combination in ['stacking', 'stacking_proba']:

            X_stacking = _create_stacking_set(self.estimators_, self.estimators_features_,
                                              self.estimators_weight_, X, self.combination)
            return self.f_staking.predict(X_stacking)

        elif self.combination in ['majority_voting', 'weighted_voting']:
            # Parallel loop
            n_jobs, n_estimators, starts = _partition_estimators(self.n_estimators,
                                                                 self.n_jobs)

            all_pred = Parallel(n_jobs=n_jobs, verbose=self.verbose)(
                delayed(_parallel_predict)(
                    self.estimators_[starts[i]:starts[i + 1]],
                    self.estimators_features_[starts[i]:starts[i + 1]],
                    X,
                    self.n_classes_,
                    self.combination,
                    self.estimators_weight_[starts[i]:starts[i + 1]])
                for i in range(n_jobs))

            # Reduce
            pred = sum(all_pred) / self.n_estimators

            return self.classes_.take(np.argmax(pred, axis=1), axis=0)

        elif self.combination in ['majority_bmr', 'weighted_bmr', 'stacking_bmr', 'stacking_proba_bmr']:
            # TODO: Add check if cost_mat == None
            X_bmr = self.predict_proba(X)
            return self.f_bmr.predict(X_bmr, cost_mat)

    def predict_proba(self, X):
        """Predict class probabilities for X.

        The predicted class probabilities of an input sample is computed as
        the mean predicted class probabilities of the base estimators in the
        ensemble. If base estimators do not implement a ``predict_proba``
        method, then it resorts to voting and the predicted class probabilities
        of a an input sample represents the proportion of estimators predicting
        each class.

        Parameters
        ----------
        X : {array-like, sparse matrix} of shape = [n_samples, n_features]
            The training input samples. Sparse matrices are accepted only if
            they are supported by the base estimator.

        Returns
        -------
        p : array of shape = [n_samples, n_classes]
            The class probabilities of the input samples. The order of the
            classes corresponds to that in the attribute `classes_`.
        """
        # Check data
        # X = check_array(X, accept_sparse=['csr', 'csc', 'coo'])  # Dont in version 0.15

        if self.n_features_ != X.shape[1]:
            raise ValueError("Number of features of the model must "
                             "match the input. Model n_features is {0} and "
                             "input n_features is {1}."
                             "".format(self.n_features_, X.shape[1]))

        # Parallel loop
        n_jobs, n_estimators, starts = _partition_estimators(
            self.n_estimators, self.n_jobs)

        all_proba = Parallel(n_jobs=n_jobs, verbose=self.verbose)(
            delayed(_parallel_predict_proba)(
                self.estimators_[starts[i]:starts[i + 1]],
                self.estimators_features_[starts[i]:starts[i + 1]],
                X,
                self.n_classes_,
                self.combination,
                self.estimators_weight_[starts[i]:starts[i + 1]])
            for i in range(n_jobs))

        # Reduce
        if self.combination in ['majority_voting', 'majority_bmr']:
            proba = sum(all_proba) / self.n_estimators
        elif self.combination in ['weighted_voting', 'weighted_bmr']:
            proba = sum(all_proba)
        elif self.combination in ['stacking', 'stacking_proba', 'stacking_bmr', 'stacking_proba_bmr']:
            X_stacking = _create_stacking_set(self.estimators_, self.estimators_features_,
                                              self.estimators_weight_, X, self.combination)
            proba = self.f_staking.predict_proba(X_stacking)

        return proba

In [121]:
class CostSensitiveRandomForestClassifier(BaggingClassifier):
    """A example-dependent cost-sensitive random forest  classifier.

    Parameters
    ----------

    n_estimators : int, optional (default=10)
        The number of base estimators in the ensemble.

    combination : string, optional (default="majority_voting")
        Which combination method to use:
          - If "majority_voting" then combine by majority voting
          - If "weighted_voting" then combine by weighted voting using the
            out of bag savings as the weight for each estimator.
          - If "stacking" then a Cost Sensitive Logistic Regression is used
            to learn the combination.
          - If "stacking_proba" then a Cost Sensitive Logistic Regression trained
            with the estimated probabilities is used to learn the combination,.
          - If "stacking_bmr" then a Cost Sensitive Logistic Regression is used
            to learn the probabilities and a BayesMinimumRisk for the prediction.
          - If "stacking_proba_bmr" then a Cost Sensitive Logistic Regression trained
            with the estimated probabilities is used to learn the probabilities,
            and a BayesMinimumRisk for the prediction.
          - If "majority_bmr" then the BayesMinimumRisk algorithm is used to make the
            prediction using the predicted probabilities of majority_voting
          - If "weighted_bmr" then the BayesMinimumRisk algorithm is used to make the
            prediction using the predicted probabilities of weighted_voting

    max_features : int, float, string or None, optional (default=None)
        The number of features to consider when looking for the best split in each tree:
          - If int, then consider `max_features` features at each split.
          - If float, then `max_features` is a percentage and
            `int(max_features * n_features)` features are considered at each
            split.
          - If "auto", then `max_features=sqrt(n_features)`.
          - If "sqrt", then `max_features=sqrt(n_features)`.
          - If "log2", then `max_features=log2(n_features)`.
          - If None, then `max_features=n_features`.

        Note: the search for a split does not stop until at least one
        valid partition of the node samples is found, even if it requires to
        effectively inspect more than ``max_features`` features.

    pruned : bool, optional (default=True)
        Whenever or not to prune the decision tree using cost-based pruning

    n_jobs : int, optional (default=1)
        The number of jobs to run in parallel for both `fit` and `predict`.
        If -1, then the number of jobs is set to the number of cores.

    verbose : int, optional (default=0)
        Controls the verbosity of the building process.

    Attributes
    ----------
    `base_estimator_`: list of estimators
        The base estimator from which the ensemble is grown.

    `estimators_`: list of estimators
        The collection of fitted base estimators.

    `estimators_samples_`: list of arrays
        The subset of drawn samples (i.e., the in-bag samples) for each base
        estimator.

    `estimators_features_`: list of arrays
        The subset of drawn features for each base estimator.

    See also
    --------
    costcla.models.CostSensitiveDecisionTreeClassifier

    References
    ----------

    .. [1] Correa Bahnsen, A., Aouada, D., & Ottersten, B.
           `"Ensemble of Example-Dependent Cost-Sensitive Decision Trees" <http://arxiv.org/abs/1505.04637>`__,
           2015, http://arxiv.org/abs/1505.04637.
    Examples
    --------
    >>> from sklearn.ensemble import RandomForestClassifier
    >>> from sklearn.cross_validation import train_test_split
    >>> from costcla.datasets import load_creditscoring1
    >>> from costcla.models import CostSensitiveRandomForestClassifier
    >>> from costcla.metrics import savings_score
    >>> data = load_creditscoring1()
    >>> sets = train_test_split(data.data, data.target, data.cost_mat, test_size=0.33, random_state=0)
    >>> X_train, X_test, y_train, y_test, cost_mat_train, cost_mat_test = sets
    >>> y_pred_test_rf = RandomForestClassifier(random_state=0).fit(X_train, y_train).predict(X_test)
    >>> f = CostSensitiveRandomForestClassifier()
    >>> y_pred_test_csdt = f.fit(X_train, y_train, cost_mat_train).predict(X_test)
    >>> # Savings using only RandomForest
    >>> print(savings_score(y_test, y_pred_test_rf, cost_mat_test))
    0.12454256594
    >>> # Savings using CostSensitiveRandomForestClassifier
    >>> print(savings_score(y_test, y_pred_test_csdt, cost_mat_test))
    0.499390945808
    """
    def __init__(self,
                 n_estimators=10,
                 combination='majority_voting',
                 max_features='auto',
                 n_jobs=1,
                 verbose=False,
                 pruned=False,
                 min_samples_leaf = 5, 
                 max_depth = 10

                ):
        super(BaggingClassifier, self).__init__(
            base_estimator=CostSensitiveDecisionTreeClassifier(max_features=max_features, pruned=pruned, min_samples_leaf = min_samples_leaf, max_depth = max_depth ),
            n_estimators=n_estimators,
            max_samples=1.0,
            max_features=1.0,
            bootstrap=True,
            bootstrap_features=False,
            combination=combination,
            n_jobs=n_jobs,
            random_state=None,
            verbose=verbose)
        self.pruned = pruned

In [126]:
model = CostSensitiveRandomForestClassifier()
help(model)

Help on CostSensitiveRandomForestClassifier in module __main__ object:

class CostSensitiveRandomForestClassifier(BaggingClassifier)
 |  CostSensitiveRandomForestClassifier(n_estimators=10, combination='majority_voting', max_features='auto', n_jobs=1, verbose=False, pruned=False, min_samples_leaf=5, max_depth=10)
 |  
 |  A example-dependent cost-sensitive random forest  classifier.
 |  
 |  Parameters
 |  ----------
 |  
 |  n_estimators : int, optional (default=10)
 |      The number of base estimators in the ensemble.
 |  
 |  combination : string, optional (default="majority_voting")
 |      Which combination method to use:
 |        - If "majority_voting" then combine by majority voting
 |        - If "weighted_voting" then combine by weighted voting using the
 |          out of bag savings as the weight for each estimator.
 |        - If "stacking" then a Cost Sensitive Logistic Regression is used
 |          to learn the combination.
 |        - If "stacking_proba" then a Cost S

# Models

## Decision Tree

In [11]:
def dt(X_train, y_train, depth, samples):
    model = DecisionTreeClassifier(min_samples_leaf = samples, max_depth = depth)
    model.fit(X_train, y_train) 
    loops = 0
    return model, loops

## Random Forest

In [12]:
def rf(X_train, y_train, depth, samples):
    model = RandomForestClassifier(min_samples_leaf = samples, max_depth = depth, n_jobs = 1)
    model.fit(X_train, y_train)
    loops = 0
    return model, loops

## Cost Sensitive Decision Tree

In [92]:
def cs(X_train, y_train, depth, samples, cost_matrix):
    model = CostSensitiveDecisionTreeClassifier() #The base object, we will train it.
    model.set_params(min_samples_leaf= samples, max_depth= depth, pruned = True, min_gain = 0.01)
    model.fit(X_train.to_numpy(), y_train.to_numpy(), prepare_for_cost_cle(len(X_train), cost_matrix))     
    loops = 0
    return model, loops

In [116]:
def AdaCSL_WRC(X_train, y_train, X_test, constraint, depth, samples ,if_cfp_equal_cfn):
    effective_threshold = 1.0 
    
    if if_cfp_equal_cfn:
        cfp = cfn = 1
    else:
        cfp = 0
        cfn = 10
    
    loops = 0
    
    for i in range(50):
        loops += 1
        cfp += 1 
        threshold = cfp/(cfp+cfn)
        print(
            f"Threshold: {threshold:.2f}", 
            f"Effective Threshold: {effective_threshold:.2f}", 
            f"Cost False Positive: {cfp:.2f}", 
            sep = " ", end="\r", flush=True)
        
        cost_matrix = np.array([[0, cfp], 
                                [cfn, 0]])
        
        model_i,_ = cs(X_train, y_train, depth, samples, cost_matrix) #create a cs model 
        
        #set the effective threshold according to the X_test 
        y_pred_proba = model_i.predict_proba(X_test.to_numpy())[:, 1]
        effective_threshold = find_effective_threshold(y_pred_proba, constraint, threshold) 
      
        #The binary search, if we overshot, we need to decrease cfp
        if effective_threshold <= threshold: 
            break

        
    best_model_ = deepcopy(model_i) #This one is already trained.
    cfp_ = cfp
    threshold_ = threshold
    
    print("\nDONE")
    print(f"Threshold: {threshold:.2f}")
    print(f"Effective threshold: {effective_threshold:.2f}")
    print("Current cfp: ", cfp)
    return best_model_ , loops

## Cost Sensitive Random Forest

In [15]:
def rf_cs(X_train, y_train, depth, samples, cost_matrix):
    model = CostSensitiveRandomForestClassifier() #The base object, we will train it.
    model.set_params(min_samples_leaf= samples, max_depth= depth, n_estimators = 100, n_jobs = -1)
    model.fit(X_train, y_train, cost_mat = prepare_for_cost_cle(len(X_train), cost_matrix))     
    loops = 0
    return model, loops

In [16]:
def constrain_rf_cs(X_train, y_train, constraint, depth, samples ,if_cfp_equal_cfn):
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    loops = 0
    return model, loops

## Help functions

In [66]:
def calc_max_acc(n_pos, const, n_cred):
    if n_pos > const:
        return (const + n_cred - n_pos)/n_cred
    else: 
        return (n_pos + n_cred - const)/n_cred

In [69]:
def create_groupby_df(mask_dates, mask_test_provider, min_const=0.9, max_const=0.7):
    groupby_df = pd.DataFrame(df[mask_dates & mask_test_provider].groupby('prediction_date')['target'].sum())
    n_cred = ((df[mask_dates & mask_test_provider].shape[0]/10))
    groupby_df = groupby_df.rename(columns={'target': 'n_positive'})
    groupby_df['%positive'] = groupby_df['n_positive']/n_cred
    constraints_list = []
    constraints_list.append(np.round(groupby_df['n_positive'].min()*min_const)) # the constraint is about 90 percent from the minimum positives in the test dates - can be changed
    constraints_list.append(np.round(groupby_df['n_positive'].max()*max_const))
    for i, constraint in enumerate(constraints_list):
        groupby_df[f'%constraint_{i}'] = constraint/groupby_df['n_positive']
    return groupby_df, constraints_list, n_cred

In [117]:
def store_results(uuid ,provider, date, max_accuracy, model_name, constraint,best_params, loops_in_model, best_model_duration, all_duration, cost, accuracy, precision, recall, f1):
    # General Results File
    results_file = 'results/all_results.csv'
    
    output = f'{uuid},{provider},{date},{max_accuracy},{model_name},{constraint},{best_params},{loops_in_model},{best_model_duration},{all_duration},{cost},{accuracy},{precision},{recall},{f1}'
    if not os.path.exists(results_file):
        with open(results_file, 'a') as output_file:  # 'a' for append to end of file
            headlines = 'uuid,provider,date,max_accuracy,model_name,constraint,best_params,loops_in_model,best_model_duration,all_duration,cost,accuracy,precision,recall,f1'
            output_file.write(headlines + "\n" + output + "\n")
    else:
        with open(results_file, 'a') as output_file:  # 'a' for append to end of file
            output_file.write(output + "\n")

## Train models
A function that trains each model according to our template

In [115]:
def model_train(X_train, y_train, X_val, y_val, X_test, constraint, model_name, cost_matrix, depths, nums_samples_leaf):
    assert model_name in ['dt', 'rf', 'cs', 'rf_cs', 'AdaCSL_WRC','constrain_rf_cs','cfn=cfp AdaCSL_WRC','cfn=cfp constrain_rf_cs']
    
    best_acc = [0,0,0] #best acc is first, second the depth and third the min sample on leaf in the model of the best acc 
    all_duration = 0
    for depth in depths:
        for samples in nums_samples_leaf:
            start = default_timer() #set start time of training
            #fit the model 
            model ,loops_in_model = {
                    'dt': lambda: dt(X_train, y_train, depth, samples),
                    'rf': lambda: rf(X_train, y_train, depth, samples),
                    'cs': lambda: cs(X_train, y_train, depth, samples, np.array([[0, 0],[10, 0]])),
                    'rf_cs': lambda: rf_cs(X_train, y_train, depth, samples,np.array([[0,0],[10, 0]])),
                    'AdaCSL_WRC': lambda: AdaCSL_WRC(X_train, y_train, X_test, constraint, depth, samples ,if_cfp_equal_cfn = False),
                    'constrain_rf_cs': lambda: constrain_rf_cs(X_train, y_train, constraint, depth, samples ,if_cfp_equal_cfn = False),
                    'cfn=cfp constrain_rf_cs': lambda: constrain_rf_cs(X_train, y_train, constraint, depth, samples ,if_cfp_equal_cfn = True),
                    'cfn=cfp AdaCSL_WRC': lambda: AdaCSL_WRC(X_train, y_train, X_test, constraint, depth, samples ,if_cfp_equal_cfn = True)
                    }[model_name]()
            
            duration = default_timer() - start #clac the duration
            all_duration += duration
            
            #evaluate validation
            y_val_pred = model.predict(X_val.to_numpy()) #val preds
            vlues = evaluate(y_val, y_val_pred, cost_matrix)
            
            if best_acc[0] < vlues['accuracy']:#changing the values for the better model
                best_acc[0] = vlues['accuracy']
                best_acc[1] = depth
                best_acc[2] = samples
                best_model = model
                best_model_duration = duration
                
    return best_model ,f'depth = {best_acc[1]} and min_smaple = {best_acc[2]}', loops_in_model ,best_model_duration , all_duration 
            

In [108]:
def model_test(model, X_test, y_test, constraint):
    y_pred_proba = model.predict_proba(X_test.to_numpy())[:, 1] #get test prediction
    y_pred_constrained = prediction_up_to_constraint(y_pred_proba, constraint) #only until the constraint
    values = evaluate(y_test, y_pred_constrained, cost_matrix) #evluate values
    return values

# Run Expriment

## Data-specific code

This part includes some data-specific code. It is not included in the final model. <br>
The X and the y are the dataframes that are used for training and testing.


###### Get credentialset Data

In [22]:
df = pd.read_csv('./data/feature_dataset_500_no_missing.csv')
df.shape

(29804, 26)

###### Get Providers Data

In [23]:
provider_ids = pickle.load(open("data\provider_ids.pickle", "rb"))

In [24]:
df_with_provider = df.join(provider_ids, on='credentialset_id')

In [138]:
providers_list = df_with_provider['provider_id'].value_counts()[3:4].index
providers_list

Index(['856f211d-c66e-4db7-b910-7419900a70e1'], dtype='object', name='provider_id')

##### Set Cost Matrix

In [139]:
COST_FALSE_NEGATIVE = 10
COST_FALSE_POSITIVE = 1
cost_matrix = np.array([[0, COST_FALSE_POSITIVE], [COST_FALSE_NEGATIVE, 0]])

## Run

In [140]:
test_dates = np.sort(df['prediction_date'].unique()[-7:])
val_dates = np.sort(df['prediction_date'].unique()[-14:-7])
mask_dates = df['prediction_date'].isin(test_dates)

In [141]:
import uuid #dont need to be here.. its in the import section i dont know why its not working 

models_name = ['dt','rf','cs','AdaCSL_WRC','cfn=cfp AdaCSL_WRC']#, 'cs', 'rf_cs', 'AdaCSL_WRC','constrain_rf_cs','cfn=cfp AdaCSL_WRC','cfn=cfp constrain_rf_cs']
depths = [3,5,10]
nums_samples_leaf = [5,15,50]

for provider in providers_list:
    uuid = str(uuid.uuid4())
    mask_test_val_provider = df_with_provider['provider_id'] == provider
    provider_groupby_df, constraints_list, n_cred = create_groupby_df(mask_dates, mask_test_val_provider)

    for i, date in enumerate(test_dates):
        #create mask dates 
        mask_train_dates = df['prediction_date'].isin(df['prediction_date'].unique()[i:-14+i])
        mask_test_date = df['prediction_date'] == date
        mask_val_date = df['prediction_date'] == val_dates[i]
        
        # creae test val and train 
        X_train = df[mask_train_dates].drop(labels=['credentialset_id', 'prediction_date', 'target'], axis=1)
        X_test = df[mask_test_val_provider & mask_test_date].drop(labels=['credentialset_id', 'prediction_date', 'target'], axis=1)
        X_val = df[mask_test_val_provider & mask_val_date].drop(labels=['credentialset_id', 'prediction_date', 'target'], axis=1)
        y_train = df[mask_train_dates]['target']
        y_test = df[mask_test_date & mask_test_val_provider]['target']
        y_val = df[mask_val_date & mask_test_val_provider]['target']
        
        #loop model
        for model_name in models_name:
            #loop constraints
            for num, constraint in enumerate(constraints_list):
                const = constraint/len(y_test)
                #create model
                model ,best_params, loops_in_model ,best_model_duration ,all_duration  = model_train(X_train, y_train, X_val, y_val, X_test, const, model_name, cost_matrix, depths, nums_samples_leaf)
                #test model
                values = model_test(model,  X_test, y_test, const)
                #best accuracy
                max_accuracy = calc_max_acc(provider_groupby_df['n_positive'][i], constraint, len(y_test))
                #store resualts
                store_results(uuid, provider, date, max_accuracy, model_name, constraint ,best_params, loops_in_model, best_model_duration, all_duration, values['cost'], values['accuracy'], values['precision'], values['recall'], values['f1'])
                

C:\Users\amith\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\amith\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\amith\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\amith\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\amith\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\amith\anaconda3\Lib\site-packages\sklearn\base.py:4

C:\Users\amith\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\amith\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\amith\AppData\Local\Temp\ipykernel_26276\381021599.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_accuracy = calc_max_acc(provider_groupby_df['n_positive'][i], constraint, len(y_test))
C:\Users\amith\AppData\Local\Temp\ipykernel_26276\381021599.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataF

C:\Users\amith\AppData\Local\Temp\ipykernel_26276\381021599.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_accuracy = calc_max_acc(provider_groupby_df['n_positive'][i], constraint, len(y_test))


Threshold: 0.71 Effective Threshold: 0.86 Cost False Positive: 24.00
DONE
Threshold: 0.71
Effective threshold: 0.71
Current cfp:  24
Threshold: 0.71 Effective Threshold: 0.86 Cost False Positive: 24.00
DONE
Threshold: 0.71
Effective threshold: 0.71
Current cfp:  24
Threshold: 0.71 Effective Threshold: 0.86 Cost False Positive: 24.00
DONE
Threshold: 0.71
Effective threshold: 0.71
Current cfp:  24
Threshold: 0.71 Effective Threshold: 0.86 Cost False Positive: 24.00
DONE
Threshold: 0.71
Effective threshold: 0.71
Current cfp:  24
Threshold: 0.71 Effective Threshold: 0.86 Cost False Positive: 24.00
DONE
Threshold: 0.71
Effective threshold: 0.71
Current cfp:  24
Threshold: 0.71 Effective Threshold: 0.86 Cost False Positive: 24.00
DONE
Threshold: 0.71
Effective threshold: 0.71
Current cfp:  24
Threshold: 0.71 Effective Threshold: 0.86 Cost False Positive: 24.00
DONE
Threshold: 0.71
Effective threshold: 0.71
Current cfp:  24
Threshold: 0.71 Effective Threshold: 0.86 Cost False Positive: 24.00


C:\Users\amith\AppData\Local\Temp\ipykernel_26276\381021599.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_accuracy = calc_max_acc(provider_groupby_df['n_positive'][i], constraint, len(y_test))


...

C:\Users\amith\AppData\Local\Temp\ipykernel_26276\381021599.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_accuracy = calc_max_acc(provider_groupby_df['n_positive'][i], constraint, len(y_test))


Threshold: 0.75 Effective Threshold: 0.83 Cost False Positive: 3.00
DONE
Threshold: 0.75
Effective threshold: 0.75
Current cfp:  3
Threshold: 0.75 Effective Threshold: 0.83 Cost False Positive: 3.00
DONE
Threshold: 0.75
Effective threshold: 0.75
Current cfp:  3
Threshold: 0.75 Effective Threshold: 0.83 Cost False Positive: 3.00
DONE
Threshold: 0.75
Effective threshold: 0.75
Current cfp:  3
Threshold: 0.75 Effective Threshold: 0.83 Cost False Positive: 3.00
DONE
Threshold: 0.75
Effective threshold: 0.75
Current cfp:  3
Threshold: 0.75 Effective Threshold: 0.83 Cost False Positive: 3.00
DONE
Threshold: 0.75
Effective threshold: 0.75
Current cfp:  3
Threshold: 0.75 Effective Threshold: 0.83 Cost False Positive: 3.00
DONE
Threshold: 0.75
Effective threshold: 0.75
Current cfp:  3
Threshold: 0.75 Effective Threshold: 0.83 Cost False Positive: 3.00
DONE
Threshold: 0.75
Effective threshold: 0.75
Current cfp:  3
Threshold: 0.75 Effective Threshold: 0.83 Cost False Positive: 3.00
DONE
Threshold:

C:\Users\amith\AppData\Local\Temp\ipykernel_26276\381021599.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_accuracy = calc_max_acc(provider_groupby_df['n_positive'][i], constraint, len(y_test))



DONE
Threshold: 0.67
Effective threshold: 0.67
Current cfp:  2
Threshold: 0.67 Effective Threshold: 1.00 Cost False Positive: 2.00
DONE
Threshold: 0.67
Effective threshold: 0.67
Current cfp:  2
Threshold: 0.67 Effective Threshold: 1.00 Cost False Positive: 2.00
DONE
Threshold: 0.67
Effective threshold: 0.67
Current cfp:  2
Threshold: 0.67 Effective Threshold: 1.00 Cost False Positive: 2.00
DONE
Threshold: 0.67
Effective threshold: 0.67
Current cfp:  2
Threshold: 0.67 Effective Threshold: 1.00 Cost False Positive: 2.00
DONE
Threshold: 0.67
Effective threshold: 0.67
Current cfp:  2
Threshold: 0.67 Effective Threshold: 1.00 Cost False Positive: 2.00
DONE
Threshold: 0.67
Effective threshold: 0.67
Current cfp:  2
Threshold: 0.67 Effective Threshold: 1.00 Cost False Positive: 2.00
DONE
Threshold: 0.67
Effective threshold: 0.67
Current cfp:  2
Threshold: 0.67 Effective Threshold: 1.00 Cost False Positive: 2.00
DONE
Threshold: 0.67
Effective threshold: 0.67
Current cfp:  2
Threshold: 0.67 Eff

C:\Users\amith\AppData\Local\Temp\ipykernel_26276\381021599.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_accuracy = calc_max_acc(provider_groupby_df['n_positive'][i], constraint, len(y_test))
